In [21]:
!pip install kaggle #pip is a de facto standard package-management system used to install and manage software packages written in Python

In [0]:
!mkdir ~/.kaggle # Easiest way to download kaggle data in Google Colab

In [23]:
!ls -al ~/ # -ls A better replacement for Python's built-in dir function with searching in mind

total 68
drwx------ 1 root root 4096 Oct  9 15:18 .
drwxr-xr-x 1 root root 4096 Oct  9 14:54 ..
-rw-r--r-- 1 root root 3106 Apr  9  2018 .bashrc
drwx------ 1 root root 4096 Oct  7 15:48 .cache
drwx------ 4 root root 4096 Oct  7 15:47 .config
drwxr-xr-x 3 root root 4096 Oct  7 15:14 .gsutil
drwxr-xr-x 1 root root 4096 Oct  7 15:46 .ipython
drwx------ 2 root root 4096 Oct  7 15:46 .jupyter
drwxr-xr-x 2 root root 4096 Oct  9 15:18 .kaggle
drwxr-xr-x 2 root root 4096 Oct  9 14:55 .keras
drwx------ 1 root root 4096 Oct  7 15:46 .local
drwxr-xr-x 3 root root 4096 Oct  7 15:46 .node-gyp
drwxr-xr-x 4 root root 4096 Oct  7 15:46 .npm
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile


In [0]:
!mv kaggle2.json ~/.kaggle/kaggle.json

In [25]:
!kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
dgomonov/new-york-city-airbnb-open-data                   New York City Airbnb Open Data                       2MB  2019-08-12 16:24:45          28305  
gustavomodelli/forest-fires-in-brazil                     Forest Fires in Brazil                              31KB  2019-08-24 16:09:16           2214  
tristan581/17k-apple-app-store-strategy-games             17K Mobile Strategy Games                            8MB  2019-08-26 08:22:16           1260  
rajeevw/ufcdata                                           UFC-Fight historical data from 1993 to 2019          2MB  2019-07-05 09:58:02           1917  
chirin/africa-economic-banking-and-systemic-crisis-data   Africa Economic, Banking

In [26]:
!kaggle datasets download paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:39<00:00, 19.1MB/s]



In [0]:

from zipfile import ZipFile

zf = ZipFile('chest-xray-pneumonia.zip', 'r')
zf.extractall()
zf.close()

There are several plotting packages available for Python, the most commonly used one being matplotlib

In [0]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

In [0]:
training_dir = './chest_xray/train'
validation_dir = './chest_xray/test'
test_dir = './chest_xray/val'


In [0]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import clear_session
from keras.optimizers import SGD
from pathlib import Path
from keras.applications import InceptionV3
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from keras import initializers, regularizers

In [0]:
# configering the images by these dimentions 
height = 299
width = height

In [32]:

conv_base = InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(height, width, 3)
)

# In this First time run there is no unlocking
conv_base.trainable = False



87916544/87910968 [==============================] - 1s 0us/step


In [33]:
# seeing a shot of the data
print('Summary')
print(conv_base.summary())

Summary
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________

In [34]:

num_classes = 2
# top layer replacement built into the construction 
x = conv_base.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x - Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer=initializers.he_normal(seed=None), kernel_regularizer=regularizers.l2(.0005))(x)
x = Dropout(0.5)(x)
# Essential to have another layer for better accuracy
x = Dense(128,activation='relu', kernel_initializer=initializers.he_normal(seed=None))(x)
x = Dropout(0.25)(x)
predictions = Dense(num_classes,  kernel_initializer="glorot_uniform", activation='softmax')(x)

print('Stacking New Layers')
model = Model(inputs = conv_base.input, outputs=predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Stacking New Layers


In [0]:
from keras.callbacks import LearningRateScheduler

# Slow down training deeper into dataset
def schedule(epoch):
    if epoch < 6:
        # Warmup model first
        return .0000032
    elif epoch < 12:
        return .01
    elif epoch < 15:
        return .002
    elif epoch < 18:
        return .0004
    elif epoch < 20:
        return .00008
    elif epoch < 80:
        return .000016
    elif epoch < 95:
        return .0000032        
    else:
        return .0000009    
      
lr_scheduler = LearningRateScheduler(schedule)

In [36]:
print('Compile model')
opt = SGD(momentum=.9)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

Compile model



In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should not be modified
validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [0]:
train_dir = os.path.join('./chest_xray', 'train')
test_dir = os.path.join('./chest_xray', 'test')

In [39]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    class_mode='categorical',
    batch_size=64
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [40]:

history = model.fit_generator(
    train_generator,
    callbacks=[lr_scheduler],
    epochs=10,
    steps_per_epoch=50,
    shuffle=True,
    workers=4,
    use_multiprocessing=False,
    validation_data=validation_generator,
    validation_steps=10
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
50/50 [==============================] - 3623s 72s/step - loss: 1.0007 - acc: 0.5397 - val_loss: 0.9411 - val_acc: 0.5946
Epoch 2/10
50/50 [==============================] - 3605s 72s/step - loss: 0.9693 - acc: 0.5666 - val_loss: 0.9383 - val_acc: 0.6170
Epoch 3/10
50/50 [==============================] - 3606s 72s/step - loss: 0.9524 - acc: 0.5812 - val_loss: 0.9356 - val_acc: 0.6250
Epoch 4/10
50/50 [==============================] - 3634s 73s/step - loss: 0.9461 - acc: 0.5972 - val_loss: 0.9326 - val_acc: 0.6218
Epoch 5/10
50/50 [==============================] - 3555s 71s/step - loss: 0.9300 - acc: 0.6141 - val_loss: 0.9332 - val_acc: 0.6250
Epoch 6/10
50/50 [==============================] - 3606s 72s/step - loss: 0.9261 - acc: 0.6169 - val_loss: 0.9341 - val_acc: 0.6250
Epoch 7/10
50/50 [==============================] - 3719s 74s/step - loss: 0.4950 - acc: 0.9009 - val_loss: 

In [41]:
# Creating the saved model for future use 
print('Saving Model')
model.save("xray.h5")

Saving Model


Validation is now made based on the new model created 

In [0]:
from keras.preprocessing import image
import numpy as np

In [43]:
for f in sorted(Path("./chest_xray/val/").glob("**/*.jpeg")):
    # Loading the image file to test the outcome
    image_to_test = image.load_img(
        str(f),
        target_size=(299, 299)
    )

    # Converting all of the image range data to a numpy array
    # now suitable for Keras interpretation Use Keras if you need a deep learning library that Allows for easy and fast prototyping
    image_to_test = image.img_to_array(image_to_test)
    # normalize to a 0 to 1 value in validation 
    image_to_test /= 255

    # Add a fourth dimension to the images 
    # Keras expects to find this in the list of images
    list_of_images = np.expand_dims(
        image_to_test,
        axis=0
    )
    # Make a prediction using the model build 
    results = model.predict(list_of_images)

    # check the first result directly as we only passed in one test image
    image_likelihood = results[0][0]

    # The result will be a number from 0.0 to 1.0
    # representing the likelihood that this
    # image is a truck.
    if image_likelihood > 0.4:
        print(f"{f} is a NORMAL! ({image_likelihood:.2f})")
    else:
        print(f"{f} is pneumonia! ({image_likelihood:.2f})")

chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg is a NORMAL! (0.93)
chest_xray/val/NORMAL/NORMAL2-IM-1430-0001.jpeg is a NORMAL! (1.00)
chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg is a NORMAL! (0.94)
chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg is a NORMAL! (1.00)
chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg is a NORMAL! (0.99)
chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg is a NORMAL! (0.95)
chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg is a NORMAL! (1.00)
chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg is a NORMAL! (1.00)
chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg is a NORMAL! (0.72)
chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg is a NORMAL! (0.90)
chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg is pneumonia! (0.02)
chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg is pneumonia! (0.33)
chest_xray/val/PNEUMONIA/person1950_bacteria_4881.jpeg is a NORMAL! (0.54)
chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg is pneumonia! (0.00)
che